In [ ]:
import os
import shutil
import SimpleITK as sitk

def has_segmentation_data(segmentation_file):
    segmentation_image = sitk.ReadImage(segmentation_file)
    segmentation_array = sitk.GetArrayFromImage(segmentation_image)
    return segmentation_array.any()

def copy_slices_with_segmentation(data_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate through each case directory
    for case_dir in os.listdir(data_dir):
        case_input_dir = os.path.join(data_dir, case_dir)
        case_output_dir = os.path.join(output_dir, case_dir)
        if not os.path.exists(case_output_dir):
            os.makedirs(case_output_dir)
        
        # Check if segmentation data is present
        segmentation_file = os.path.join(case_input_dir, 'segmentation.nii.gz')
        if has_segmentation_data(segmentation_file):
            # Create folders for CT scans and segmentation scans
            ct_scan_dir = os.path.join(case_output_dir, 'CT_scans')
            if not os.path.exists(ct_scan_dir):
                os.makedirs(ct_scan_dir)
                
            segmentation_dir = os.path.join(case_output_dir, 'segmentation_scans')
            if not os.path.exists(segmentation_dir):
                os.makedirs(segmentation_dir)
            
            # Copy original segmentation file to new directory
            shutil.copy(segmentation_file, segmentation_dir)
            
            # Copy CT scan slices with segmentation data
            imaging_file = os.path.join(case_input_dir, 'imaging.nii.gz')
            original_imaging_image = sitk.ReadImage(imaging_file)
            original_segmentation_image = sitk.ReadImage(segmentation_file)
            
            for idx in range(original_imaging_image.GetDepth()):
                segmentation_slice = original_segmentation_image[:,:,idx]
                if sitk.GetArrayViewFromImage(segmentation_slice).any():
                    imaging_slice = original_imaging_image[:,:,idx]
                    
                    # Create copies of the slices
                    imaging_slice_copy = sitk.Image(imaging_slice.GetSize(), imaging_slice.GetPixelID())
                    imaging_slice_copy.CopyInformation(imaging_slice)
                    imaging_slice_copy = sitk.Paste(imaging_slice_copy, imaging_slice, destinationIndex=(0,0,idx))
                    
                    segmentation_slice_copy = sitk.Image(segmentation_slice.GetSize(), segmentation_slice.GetPixelID())
                    segmentation_slice_copy.CopyInformation(segmentation_slice)
                    segmentation_slice_copy = sitk.Paste(segmentation_slice_copy, segmentation_slice, destinationIndex=(0,0,idx))
                    
                    # Save the copied slices
                    imaging_slice_path = os.path.join(ct_scan_dir, f'imaging_slice_{idx}.nii.gz')
                    segmentation_slice_path = os.path.join(segmentation_dir, f'segmentation_slice_{idx}.nii.gz')
                    sitk.WriteImage(imaging_slice_copy, imaging_slice_path)
                    sitk.WriteImage(segmentation_slice_copy, segmentation_slice_path)
                    
                    # Print the paths of saved slices for debugging
                    print(f"Saved CT scan slice: {imaging_slice_path}")
                    print(f"Saved segmentation slice: {segmentation_slice_path}")
            
            print(f"Slices copied for case {case_dir}")
        else:
            print(f"No segmentation data found for case {case_dir}. Skipping...")

# Usage:
data_dir = r"C:\Users\ganess4\Desktop\kits23\dataset"
output_dir = r"C:\Users\ganess4\Desktop\kits23\processed_data_1"
copy_slices_with_segmentation(data_dir, output_dir)


In [ ]:
#Second Step of Preprocessing: Reducing the Resolution
# Load the imaging and segmentation files
imaging_path = r"C:\Users\ganess4\Desktop\kits23\dataset\case_00002\imaging.nii.gz"
segmentation_path = r"C:\Users\ganess4\Desktop\kits23\dataset\case_00002\segmentation.nii.gz"

imaging_nii = nib.load(imaging_path)
segmentation_nii = nib.load(segmentation_path)

# Extract the data arrays
imaging_data = imaging_nii.get_fdata()
segmentation_data = segmentation_nii.get_fdata()

# Check the shape and print it for debugging purposes
print("Imaging shape:", imaging_data.shape)
print("Segmentation shape:", segmentation_data.shape)

# Ensure the width and height are the same for imaging and segmentation data
assert imaging_data.shape[1] == imaging_data.shape[2], "Width and height do not match for imaging data"
assert segmentation_data.shape[1] == segmentation_data.shape[2], "Width and height do not match for segmentation data"

# Find indices of slices with segmentation (non-zero pixels)
# The first dimension is assumed to be the slice dimension
segmented_slices = np.any(segmentation_data, axis=(1, 2))

# Filter out slices without segmentation from both imaging and segmentation data
imaging_data_filtered = imaging_data[segmented_slices, :, :]
segmentation_data_filtered = segmentation_data[segmented_slices, :, :]

# Reduce resolution to 128x128
imaging_data_resized = np.zeros((imaging_data_filtered.shape[0], 128, 128))
segmentation_data_resized = np.zeros((segmentation_data_filtered.shape[0], 128, 128))

for i in range(imaging_data_filtered.shape[0]):
    imaging_data_resized[i] = resize(imaging_data_filtered[i], (128, 128), anti_aliasing=True)
    segmentation_data_resized[i] = resize(segmentation_data_filtered[i], (128, 128), anti_aliasing=False)

# Create new NIfTI objects for the resized data
imaging_nii_resized = nib.Nifti1Image(imaging_data_resized, imaging_nii.affine)
segmentation_nii_resized = nib.Nifti1Image(segmentation_data_resized, segmentation_nii.affine)

# Define paths for the new resized files
output_dir = r"C:\Users\ganess4\Desktop\kits23\processed_data_1"
os.makedirs(output_dir, exist_ok=True)

resized_imaging_path = os.path.join(output_dir, 'case_00002_imaging_filtered.nii.gz')
resized_segmentation_path = os.path.join(output_dir, 'case_00002_segmentation_filtered.nii.gz')

print("Resized Imaging shape:", imaging_data_resized.shape)
print("Resized Segmentation shape:", segmentation_data_resized.shape)

# Save the resized NIfTI files
nib.save(imaging_nii_resized, resized_imaging_path)
nib.save(segmentation_nii_resized, resized_segmentation_path)